In [1]:
#import libraries
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#import dataframes
cusdemo_df = pd.read_excel(r"C:\Users\Galvin\Desktop\KPMG Virtual Internship\KPMG_VI_New_raw_data_update_final.xlsx",
                          sheet_name=3,
                          header=1,
                          skiprows=0,
                          )

cusadd_df = pd.read_excel(r"C:\Users\Galvin\Desktop\KPMG Virtual Internship\KPMG_VI_New_raw_data_update_final.xlsx",
                          sheet_name=4,
                          header=1,
                          skiprows=0,
                          )

newcus_df = pd.read_excel(r"C:\Users\Galvin\Desktop\KPMG Virtual Internship\KPMG_VI_New_raw_data_update_final.xlsx",
                          sheet_name=2,
                          header=1,
                          skiprows=0,
                          )

In [3]:
#merge dataframes
oldcus_df = pd.merge(cusdemo_df, cusadd_df, on=['customer_id'])

In [4]:
#drop redundant columns
oldcus_df.drop(columns=['customer_id', 'first_name', 'last_name', 'deceased_indicator', 'address', 'country'], inplace=True)
newcus_df.drop(columns=['first_name', 'last_name', 'deceased_indicator', 'address', 'country'], inplace=True)

In [5]:
#check dimensions
oldcus_df.shape, newcus_df.shape

((3496, 12), (1000, 11))

In [6]:
#fill blanks with values
oldcus_df.fillna(oldcus_df.mean(), inplace=True)
oldcus_df.fillna({'gender':'Female', 'job_title':'Business Systems Development Analyst', 
                         'job_industry_category':'Manufacturing', 'DOB':'1978-01-30'}, inplace=True)

newcus_df.fillna(newcus_df.mean(), inplace=True)
newcus_df.fillna({'gender':'Female', 'job_title':'Associate Professor', 
                         'job_industry_category':'Financial Services', 'DOB':'1961-07-31'}, inplace=True)

In [7]:
#check completeness
oldcus_df.isnull().sum()

gender                                 0
past_3_years_bike_related_purchases    0
DOB                                    0
job_title                              0
job_industry_category                  0
wealth_segment                         0
owns_car                               0
tenure                                 0
postcode                               0
state                                  0
property_valuation                     0
total_revenue                          0
dtype: int64

In [8]:
#check completeness
newcus_df.isnull().sum()

gender                                 0
past_3_years_bike_related_purchases    0
DOB                                    0
job_title                              0
job_industry_category                  0
wealth_segment                         0
owns_car                               0
tenure                                 0
postcode                               0
state                                  0
property_valuation                     0
dtype: int64

In [9]:
#convert DOB to age
old_year = pd.DatetimeIndex(oldcus_df['DOB']).year
new_year = pd.DatetimeIndex(newcus_df['DOB']).year
old_age = 2018 - old_year
new_age = 2018 - new_year
oldcus_df.insert(2,column='age', value=old_age)
newcus_df.insert(2,column='age', value=new_age)
oldcus_df.drop(columns=['DOB'], inplace=True)
newcus_df.drop(columns=['DOB'], inplace=True)

In [10]:
#dataframe of categorical variables
oldcus_df_cat = oldcus_df[['gender', 'job_title', 'job_industry_category', 'wealth_segment', 'owns_car', 'state']]
newcus_df_cat = newcus_df[['gender', 'job_title', 'job_industry_category', 'wealth_segment', 'owns_car', 'state']]

In [11]:
#encode categorical variables
le = LabelEncoder()
oldcus_df_cat_enc = oldcus_df_cat.apply(le.fit_transform)
newcus_df_cat_enc = newcus_df_cat.apply(le.fit_transform)

In [12]:
#create encoded dataframe
oldcus_df[['gender', 'job_title', 'job_industry_category', 'wealth_segment', 'owns_car', 'state']]  = oldcus_df_cat_enc[['gender', 'job_title', 'job_industry_category', 'wealth_segment', 'owns_car', 'state']]
newcus_df[['gender', 'job_title', 'job_industry_category', 'wealth_segment', 'owns_car', 'state']]  = newcus_df_cat_enc[['gender', 'job_title', 'job_industry_category', 'wealth_segment', 'owns_car', 'state']]

In [13]:
#categorise prediction variable
bin_labels = ['L', 'M', 'H', 'VH']
oldcus_df['total_revenue_enc'] = pd.qcut(oldcus_df['total_revenue'],
                              q=4,
                              labels=bin_labels)

In [20]:
#check correlation
oldcus_df.corr()['total_revenue']

gender                                 0.007438
past_3_years_bike_related_purchases    0.003122
age                                   -0.005675
job_title                             -0.011971
job_industry_category                  0.008362
wealth_segment                        -0.004060
owns_car                              -0.009989
tenure                                 0.009486
postcode                               0.009573
state                                  0.002509
property_valuation                    -0.014971
total_revenue                          1.000000
Name: total_revenue, dtype: float64

In [15]:
#split data features
x_train = oldcus_df.drop(columns=['total_revenue', 'total_revenue_enc'])
y_train = oldcus_df['total_revenue_enc']

In [16]:
#train and evaluate model
dectree = DecisionTreeClassifier()
dectree.fit(x_train, y_train)
acc_dectree = dectree.score(x_train, y_train)
acc_dectree

1.0

In [17]:
#make predictions
pred = dectree.predict(newcus_df)
pred

array(['L', 'L', 'H', 'VH', 'VH', 'VH', 'VH', 'VH', 'M', 'L', 'L', 'L',
       'M', 'M', 'VH', 'L', 'H', 'M', 'H', 'M', 'VH', 'VH', 'L', 'L', 'L',
       'VH', 'L', 'VH', 'VH', 'H', 'L', 'H', 'M', 'L', 'M', 'VH', 'M',
       'M', 'L', 'L', 'L', 'M', 'VH', 'H', 'L', 'M', 'M', 'H', 'L', 'H',
       'L', 'L', 'L', 'H', 'L', 'H', 'M', 'H', 'VH', 'H', 'H', 'M', 'H',
       'M', 'H', 'L', 'VH', 'M', 'L', 'M', 'H', 'L', 'M', 'VH', 'VH', 'L',
       'L', 'H', 'VH', 'M', 'H', 'H', 'H', 'VH', 'VH', 'VH', 'L', 'VH',
       'H', 'H', 'L', 'VH', 'VH', 'M', 'M', 'M', 'M', 'VH', 'H', 'M', 'H',
       'VH', 'H', 'L', 'H', 'M', 'H', 'M', 'M', 'H', 'H', 'VH', 'M', 'H',
       'H', 'M', 'VH', 'H', 'VH', 'L', 'L', 'L', 'L', 'L', 'VH', 'M', 'L',
       'VH', 'L', 'L', 'VH', 'L', 'H', 'M', 'VH', 'H', 'M', 'L', 'H', 'H',
       'L', 'VH', 'M', 'L', 'M', 'L', 'M', 'L', 'H', 'H', 'M', 'VH', 'M',
       'VH', 'H', 'H', 'H', 'M', 'H', 'L', 'M', 'L', 'M', 'L', 'VH', 'M',
       'L', 'L', 'H', 'VH', 'L', 'M', 'H',

In [18]:
#merge prediction with new customer dataframe
newcus_df['prediction'] = pred
newcus_df

,gender,past_3_years_bike_related_purchases,age,job_title,job_industry_category,wealth_segment,owns_car,tenure,postcode,state,property_valuation,prediction
0,1,86,61,71,5,2,1,14,4500,1,6,L
1,1,69,48,165,6,2,0,16,2113,0,11,L
2,0,10,44,139,2,0,0,10,3505,2,5,H
3,0,64,39,4,5,0,1,5,4814,1,1,VH
4,0,34,53,68,2,0,0,19,2093,0,9,VH
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,60,59,108,2,0,0,9,2200,0,7,H
996,1,22,17,144,3,2,0,6,2196,0,10,VH
997,0,17,64,32,2,0,1,15,4702,1,2,M
998,1,30,66,67,2,2,1,19,4215,1,2,M


In [29]:
#save predictions to excel file
prediction = pd.DataFrame({'prediction':pred})
prediction.to_excel(r"C:\Users\Galvin\Desktop\KPMG Virtual Internship\KPMG_VI_Predictions.xlsx", index=False)